In [0]:
import os
os.getcwd()

'/content'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dir = os.path.join('drive','My Drive','Colab Notebooks','STAT628_module3')
dir

'drive/My Drive/Colab Notebooks/STAT628_module3'

## Import packages and dataset

In [0]:
import numpy as np
import pandas as pd
import json
import string
import re

In [0]:
data = []
with open(os.path.join(dir,'dataset','business.json'), 'r') as f:
    for line in f.readlines():
      dic = json.loads(line)
      data.append(dic)
data = pd.DataFrame.from_dict(data)

## select attributes in business data

In [0]:
data = data[-data['categories'].isnull()]
filt = filter(lambda x: True if 'ice cream' in data.loc[x,'categories'].lower() else False, data.index.values)
data = data.loc[list(filt),:]
data.shape

(2764, 14)

In [0]:
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
33,irft4YkdNsww4DNf_Aftew,So Cool Frozen Yogurt,9020 B Albemarle Rd,Charlotte,NC,28227,35.210510,-80.690262,4.5,6,0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Juice Bars & Smoothies, Food, Ice Cream & Froz...","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
47,GWO87Y-IqL54_Ijx6hTYAQ,Bruster's Real Ice Cream,709 W Ray Rd,Gilbert,AZ,85233,33.320386,-111.804652,4.5,57,0,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Food, Desserts, Juice Bars & Smoothies, Ice Cr...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'..."
75,kANF0dbeoW34s2vwh6Umfw,Dairy Queen,6125 Spring Mountain Rd,Las Vegas,NV,89146,36.125031,-115.225620,2.0,33,0,"{'RestaurantsPriceRange2': '1', 'Ambience': '{...","Fast Food, Food, Restaurants, Ice Cream & Froz...",None
125,Kxbn0XLcCgQqmkdIkhuOeQ,Not Your Mother,"1346 Queen Street, Unit 2",Toronto,ON,M6K 1L4,43.641708,-79.432192,4.0,8,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","Desserts, Ice Cream & Frozen Yogurt, Food, Res...","{'Monday': '12:0-23:0', 'Tuesday': '12:0-23:0'..."
196,6aOnrzf15RMqFNOQ-_ElIA,COCO Crafted Organic Chocolate,365 Jane Street,Toronto,ON,M6S 2E2,43.657600,-79.487683,5.0,15,1,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","Chocolatiers & Shops, Specialty Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."


## preprocessing on hours

In [0]:
type(data['hours'][196])

dict

In [0]:
data = data[['business_id','stars','review_count','attributes','hours']]
data.index = pd.RangeIndex(len(data.index))
data.head()

,business_id,stars,review_count,attributes,hours
0,irft4YkdNsww4DNf_Aftew,4.5,6,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
1,GWO87Y-IqL54_Ijx6hTYAQ,4.5,57,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'..."
2,kANF0dbeoW34s2vwh6Umfw,2.0,33,"{'RestaurantsPriceRange2': '1', 'Ambience': '{...",None
3,Kxbn0XLcCgQqmkdIkhuOeQ,4.0,8,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","{'Monday': '12:0-23:0', 'Tuesday': '12:0-23:0'..."
4,6aOnrzf15RMqFNOQ-_ElIA,5.0,15,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."


In [0]:
nontime = {'Friday': None,
 'Monday': None,
 'Saturday': None,
 'Sunday': None,
 'Thursday': None,
 'Tuesday': None}
nonatt = {'NoAttributes': True}

In [0]:
ice = data.copy()
for i in range(len(ice)):
  if ice['hours'][i] == None:
    ice['hours'][i] = nontime
  if ice['attributes'][i] == None:
    ice['attributes'][i] = nonatt
ice.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,business_id,stars,review_count,attributes,hours
0,irft4YkdNsww4DNf_Aftew,4.5,6,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
1,GWO87Y-IqL54_Ijx6hTYAQ,4.5,57,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'..."
2,kANF0dbeoW34s2vwh6Umfw,2.0,33,"{'RestaurantsPriceRange2': '1', 'Ambience': '{...","{'Friday': None, 'Monday': None, 'Saturday': N..."
3,Kxbn0XLcCgQqmkdIkhuOeQ,4.0,8,"{'RestaurantsPriceRange2': '1', 'RestaurantsTa...","{'Monday': '12:0-23:0', 'Tuesday': '12:0-23:0'..."
4,6aOnrzf15RMqFNOQ-_ElIA,5.0,15,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."


In [0]:
time = list(ice['hours'])
time = pd.DataFrame.from_dict(time)
time = time.where((pd.notnull(time)), None)
time.head(20)

,Monday,Tuesday,Thursday,Friday,Saturday,Sunday,Wednesday
0,11:30-20:30,11:30-20:30,11:30-20:30,11:30-21:0,11:30-21:0,11:30-21:0,None
1,12:0-21:0,12:0-21:0,12:0-21:0,12:0-22:0,12:0-22:0,12:0-21:0,12:0-21:0
2,None,None,None,None,None,None,None
3,12:0-23:0,12:0-23:0,12:0-23:0,12:0-23:0,12:0-23:0,12:0-23:0,12:0-23:0
4,0:0-0:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,12:0-17:0,10:0-18:0
5,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0
6,9:0-19:0,9:0-19:0,9:0-19:0,9:0-19:0,10:0-19:0,None,20:0-23:0
7,12:0-21:0,12:0-21:0,12:0-21:0,12:0-22:0,12:0-22:0,12:0-21:0,12:0-21:0
8,11:30-22:0,11:30-22:0,11:30-22:0,11:30-22:0,11:30-22:0,11:30-22:0,11:30-22:0
9,9:0-3:0,9:0-3:0,9:0-3:0,9:0-3:0,11:0-3:0,11:0-3:0,9:0-3:0


In [0]:
def set_time():
  OpenTime = []
  for i in range(len(time)):
    temp = time.loc[i]
    opentime = [False, False, False]
    for day in time.columns:
      if temp[day] is None: 
        # opentime = [None, None, None]
        continue
      else:
        t = temp[day].replace(':',' ').replace('-',' ').split(' ')
        if(t[2]=='0'): t[2] = '24'
        if (int(t[0]) in range(1,11)) and (int(t[2]) > 12):
          opentime[0] = True
        if (int(t[0]) < 14 ) and (int(t[2]) > 17):
          opentime[1] = True
        if (int(t[0]) < 18) and (int(t[2]) > 19):
          opentime[2] = True
    OpenTime.append(opentime)
  return pd.DataFrame(OpenTime, columns=['Morning','Afternoon','Evening'])

In [0]:
OpenTime = set_time()

In [0]:
OpenTime.head()

,Morning,Afternoon,Evening
0,False,True,True
1,False,True,True
2,False,False,False
3,False,True,True
4,True,True,True


## preprocessing on attributes

In [0]:
att = list(ice['attributes'])
att = pd.DataFrame.from_dict(att)
att = att.where((pd.notnull(att)), None)
att = att.replace('None', None)
att.head()

,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,BikeParking,WiFi,Caters,BusinessParking,WheelchairAccessible,Ambience,RestaurantsGoodForGroups,NoiseLevel,RestaurantsAttire,OutdoorSeating,HasTV,RestaurantsReservations,RestaurantsDelivery,GoodForKids,Alcohol,RestaurantsTableService,GoodForMeal,NoAttributes,DogsAllowed,BusinessAcceptsBitcoin,ByAppointmentOnly,DriveThru,DietaryRestrictions,Smoking,GoodForDancing,BYOBCorkage,Corkage,HappyHour,CoatCheck,Music,BestNights,RestaurantsCounterService
0,True,2,True,False,'free',False,"{'garage': False, 'street': False, 'validated'...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,True,2,False,True,'no',True,"{'garage': False, 'street': False, 'validated'...",True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,True,1,True,True,'no',False,"{'garage': False, 'street': False, 'validated'...",None,"{'romantic': False, 'intimate': False, 'classy...",True,u'average',u'casual',False,False,False,False,True,u'none',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,1,True,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,2,True,True,None,True,"{'garage': False, 'street': True, 'validated':...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [0]:
s = list(ice['attributes'])
s = pd.DataFrame.from_dict(s)
p = s['BusinessParking']
type(p)

pandas.core.series.Series

In [0]:
remove = string.punctuation
remove = remove.replace(',','')
def convert_parking(parking):
  clean_parking = []
  for p in parking:
    if p is None:
      clean_parking.append(None)
      continue
    else:
      p = re.sub("[{}]+".format(remove), '',p)
      p = re.sub(' +',",",p)
      p = p.split(',')
      if 'True' in p: 
        clean_parking.append('True')
      else:
        clean_parking.append('False')
  return clean_parking 

In [0]:
p = convert_parking(att['BusinessParking'].tolist())
att['BusinessParking'] = p

In [0]:
selected_att = att.columns[att.isnull().sum()<1000].tolist()
selected_att

['BusinessAcceptsCreditCards',
 'RestaurantsPriceRange2',
 'RestaurantsTakeOut',
 'BikeParking',
 'WiFi',
 'BusinessParking']

In [0]:
att = att[selected_att]
att.head()

,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,BikeParking,WiFi,BusinessParking
0,True,2,True,False,free,False
1,True,2,False,True,no,False
2,True,1,True,True,no,True
3,None,1,True,None,None,None
4,None,2,True,True,None,True


In [0]:
# att['BusinessAcceptsCreditCards'].unique()
# att['RestaurantsPriceRange2'].unique()
# att['RestaurantsTakeOut'].unique()
# att['BikeParking'].unique()
# att['WiFi'].unique()
# att['BusinessParking'].unique()

In [0]:
att['WiFi'] = att['WiFi'].replace('u\'free\'', 'free').replace('\'free\'','free').replace('u\'no\'','no').replace('\'no\'','no').replace('u\'paid\'','paid').replace('\'paid\'','paid')

In [0]:
att['WiFi'].unique()

array(['free', 'no', None, 'paid'], dtype=object)

## Selected attributes 

In [0]:
final_att = pd.concat([ice[['business_id','stars','review_count']],OpenTime,att], axis=1)
final_att.head()

,business_id,stars,review_count,Morning,Afternoon,Evening,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsTakeOut,BikeParking,WiFi,BusinessParking
0,irft4YkdNsww4DNf_Aftew,4.5,6,False,True,True,True,2,True,False,free,False
1,GWO87Y-IqL54_Ijx6hTYAQ,4.5,57,False,True,True,True,2,False,True,no,False
2,kANF0dbeoW34s2vwh6Umfw,2.0,33,False,False,False,True,1,True,True,no,True
3,Kxbn0XLcCgQqmkdIkhuOeQ,4.0,8,False,True,True,None,1,True,None,None,None
4,6aOnrzf15RMqFNOQ-_ElIA,5.0,15,True,True,True,None,2,True,True,None,True


In [0]:
final_att.to_csv(os.path.join(dir,'dataset','final_att.csv'),index = False)